In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)#直方图


#tensorboard-命名空间
with tf.name_scope('input'):
    #设置变量
    x=tf.placeholder(tf.float32,[None,784],name='x_input')
    y=tf.placeholder(tf.float32,[None,10],name='y_input')
    

with tf.name_scope('layer'):
    with tf.name_scope('wights'):
        Weights=tf.Variable(tf.zeros([784,10]),name='Weights')
        variable_summaries(Weights)
    with tf.name_scope('bias'):
        biases=tf.Variable(tf.zeros([10]),name='bias')
        variable_summaries(biases)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b=tf.matmul(x,Weights)+biases

    #构造模型
    with tf.name_scope('prediction'):
        prediction=tf.nn.softmax(wx_plus_b)

#loss
with tf.name_scope('loss'):
    loss=tf.square(prediction-y)
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    
    
#评估
with tf.name_scope('accuracy'):
    #correct_prediction是一个bool list
    with tf.name_scope('correct_prediction'):
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        #准确率
        """
        tf.cast:转化类型 True->1.0;False->0.0
        tf.reduce_mean:求平均值，即准确率
        """
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

#合并所有summary
merged=tf.summary.merge_all()
        

sess=tf.Session()
sess.run(tf.global_variables_initializer())

#tenforboard-写入文件
writer=tf.summary.FileWriter('logs/',sess.graph)

"""
等程序运行完毕在终端执行一下命令启动tersorboard：
tensorboard --logdir=./logs/
"""

#训练
for i in range(100):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    
    summary, _  = sess.run([merged,train_step], feed_dict={x: batch_xs, y: batch_ys})
    #训练的时候将summary加入到writer
    writer.add_summary(summary,i)
    
    if i%10==0:
        res=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print(res)

writer.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'input/y_input' with dtype float and shape [?,10]
	 [[Node: input/y_input = Placeholder[dtype=DT_FLOAT, shape=[?,10], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input/y_input', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", 